First we import required modules and load the data from the .json file to a Pandas DataFrame. We inspect the DataFrame object to determine its properties. 

In [2]:
import pandas as pd
from datetime import datetime
data = pd.read_json("Assignment_03/reviews.json") # Data import 
data['date'] = data['date'].apply(lambda x: datetime.fromordinal(x)) # Date conversion
data.head(4)

,author,body,date,helpful,stars,style,title
0,Gene H.,We were looking for a decent flashlight for ou...,2018-04-11,0,5,LED Flashlight 2P,Great range for a small package
1,Cwazy Wabbit,"Very nice, works great",2018-04-11,0,5,LED Flashlight 2P,Five Stars
10,David C. Bradford,Sturdy little lights that work every time as a...,2018-04-10,0,5,LED Flashlight 2P,Sturdy little lights that work every time as
100,MDD,I bought 2 and they worked great for 2-3 weeks...,2018-03-09,0,1,LED Flashlight 2P,I bought 2 and they worked great for 2-3 weeks...


The dataframe is of the shape `(833, 7)` and its axes are: `['author', 'body', 'date', 'helpful', 'stars', 'style', 'title']`

,author,body,date,helpful,stars,style,title
0,Gene H.,We were looking for a decent flashlight for ou...,736795,0,5,LED Flashlight 2P,Great range for a small package
1,Cwazy Wabbit,"Very nice, works great",736795,0,5,LED Flashlight 2P,Five Stars
10,David C. Bradford,Sturdy little lights that work every time as a...,736794,0,5,LED Flashlight 2P,Sturdy little lights that work every time as
100,MDD,I bought 2 and they worked great for 2-3 weeks...,736762,0,1,LED Flashlight 2P,I bought 2 and they worked great for 2-3 weeks...
101,Eric,Very bright flashlight. Used at the beach walk...,736757,0,5,LED Flashlight 2P,Good flashlights for the price


The dataframe is of the shape `(833, 7)` and its axes are: `['author', 'body', 'date', 'helpful', 'stars', 'style', 'title']`